In [ ]:
def calc_distance(a,b):
    """레벤슈타인 거리 계산하기"""
    if a == b: return 0
    a_len = len(a)
    b_len = len(b)
    if a == "": return b_len
    if b == "": return a_len
    # 2차원 표 (a_len+1, b_len+1) 준비하기
    matrix = [[] for i in range(a_len+1)]
    for i in range(a_len+1):    # 0으로 초기화
        matrix[i] = [0 for j in range(b_len+1)]
    # 0일 때 초깃값을 설정
    for i in range(a_len+1):
        matrix[i][0] = i
    for j in range(b_len+1):
        matrix[0][j] = j
    # 표 채우기
    for i in range(1, a_len+1):
        ac = a[i-1]
        for j in range(1, b_len+1):
            bc = b[j-1]
            cost = 0 if (ac==bc) else 1
            matrix[i][j] = min([
                matrix[i-1][j] + 1, # 문자삽입
                matrix[i][j-1] + 1, # 문자제거
                matrix[i-1][j-1] + cost # 문자변경
            ])
    return matrix[a_len][b_len]

# "가나다라"와 "가마바라" 의 거리
print(calc_distance("가나다라","가마바라"))

# 실행 예
samples = ["신촌역", "신천군", "신천역", "신발", "마곡역"]
base = samples[0]
r = sorted(samples, key=lambda n: calc_distance(base,n))
for n in r:
    print(calc_distance(base,n),n)

In [1]:
def ngram(s, num):
    res = []
    slen = len(s) - num + 1
    for i in range(slen):
        ss = s[i:i+num]
        res.append(ss)
    return res

def diff_ngram(sa,sb,num):
    a = ngram(sa,num)
    b = ngram(sb,num)
    r = []
    cnt = 0
    for i in a:
        for j in b:
            if i == j:
                cnt += 1
                r.append(i)
    return cnt/len(a), r  

a = "오늘 강남에서 맛있는 스파게티를 먹었다."              
b = "강남에서 먹었던 오늘의 스파게티는 맛있었다."

# 2-gram
r2,word2 = diff_ngram(a,b,2)
print('2-gram : ',r2,word2)

# 3-gram
r3,word3 = diff_ngram(a,b,3)
print('3-gram : ',r3,word3)

2-gram :  0.7619047619047619 ['오늘', '강남', '남에', '에서', '서 ', ' 맛', '맛있', '는 ', ' 스', '스파', '파게', '게티', ' 먹', '먹었', '었다', '다.']
3-gram :  0.45 ['강남에', '남에서', '에서 ', ' 맛있', ' 스파', '스파게', '파게티', ' 먹었', '었다.']


In [3]:
# 파이썬을 이용한 맞춤법 처리
# 참조: https://github.com/spacedev-official/py-aiohanspell
# 모듈설치 : py-aiohanspell
#!pip install py-aiohanspell
result = await spell_checker.check(u'안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.')
result.as_dict

<bound method Checked.as_dict of Checked(result=True, original='안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.', checked='안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.', errors=4, words=OrderedDict([('안녕하세요.', 2), ('저는', 0), ('한국인입니다.', 2), ('이', 2), ('문장은', 2), ('한글로', 0), ('작성됐습니다.', 1)]), time=0.10739994049072266)>

In [7]:
## 마르코프 체인
# 순서
# 1. 문장을 단어로 분할(형태소 분석)합니다
# 2. 단어의 전후 연결을 딕셔너리에 등록합니다.
# 3. 사전을 사용해 임의 문장을 생성합니다
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
import urllib.request
import os, re, json, random
from aiohanspell import spell_checker

# 마르코프 체인 딕셔너리 만들기
def make_dic(words):
    tmp = ["@"]
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3: continue
        if len(tmp) > 3: tmp = tmp[1:]
        set_words3(dic, tmp)
        if word ==".":
            tmp = ["@"]
            continue
    return dic

# 딕셔너리에 데이터 등록하기
def set_words3(dic,s3):
    w1, w2, w3 = s3
    if not w1 in dic: dic[w1] = {}
    if not w2 in dic[w1]: dic[w1][w2] = {}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0

# 문장만들기
def make_sentence(dic):
    ret = []
    if not "@" in dic: return "no dic"
    top = dic["@"]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == ".": break
        w1,w2 = w2,w3
    ret="".join(ret)
    return ret

def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))
    
# 문장 읽기(main)
toji_file = "data_text/toji.txt"
dict_file = "data_text/markov-toji.json"
if not os.path.exists((dict_file)):
    # 토지 텍스트 파일 읽어오기
    fp = codecs.open("data_text/BEXX0003.txt","r",encoding="utf-16")
    soup = BeautifulSoup(fp,"html.parser")
    body = soup.select_one("body > text")
    text = body.getText()
    text = text.replace("…","") # 형재 konlyPy가 구두점으로 잡지 못하는 문제 임시해결
    # 형태소 분석
    twitter = Twitter()
    malist = twitter.pos(text, norm=True)
    words = []
    for word in malist:
        # 구두점 등은 대상에서 제외(단, 마침표는 포함)
        if not word[1] in ['Punctuation']:
            words.append(word[0])
        if word[0] == ".":
            words.append(word[0])
    #딕셔너리 생성
    dic = make_dic(words)
    json.dump(dic,open(dict_file,'w',encoding='utf-8'))
else:
    dic = json.load(open(dict_file,'r'))

# 문장 만들기
for i in range(3):
    s = make_sentence(dic)
    t = await spell_checker.check(s)
    print(s)
    print("-"*20)

며칠동안상식上食때면여느아낙들같이서럽게생각지마라
평산의큰아들거복이었다.
--------------------
파리가말꼬리에붙어서천리간다하기는하더라마는마음을얼어붙게했다.
--------------------
저만큼논둑길에
말을끊은그의태도는전보다더부지런히들일하고하는데멀그래쌌노
그리안하믄우쩔기요.
--------------------


In [8]:
from pprint import pprint
import json
dic = json.load(open("data_text/markov-toji.json",'r'))
pprint(dic['걱정'])

{'들': {'을': 0},
 '마라': {'\n': 0, '.': 0},
 '말': {'게': 0},
 '말고': {'마저': 0, '처': 0},
 '은': {'마라': 0, '안': 0},
 '을': {'다': 0},
 '이': {'없을': 0, '요': 0, '있고': 0, '있더라고': 0},
 '이고': {'\n': 0, '송장': 0},
 '이다': {'그': 0},
 '이며': {'이윽고': 0},
 '이제': {'.': 0},
 '일세': {'.': 0},
 '하실': {'것': 0},
 '할': {'것': 0}}


In [2]:
## LSTM
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys

fp = codecs.open("data_text/BEXX0003.txt",'r',encoding='utf-16')
soup = BeautifulSoup(fp,'html.parser')
body = soup.select_one('body')
text = body.getText() + " "
print("코퍼스의 길이 : ",len(text))

코퍼스의 길이 :  311682


In [3]:
## 문자를 하나하나 읽어 들이고 id붙이기
chars = sorted(list(set(text)))
print("사용되고  있는 문자의 수 : ",len(chars))
char_indices = dict((c,i) for i,c in enumerate(chars))  # 문자 -> id
indices_char = dict((i,c) for i,c in enumerate(chars))  # id -> 문자

사용되고  있는 문자의 수 :  1692


In [4]:
## 텍스트를 maxlen개의 문자로 자르고, 다음에 오는 문자 등록하기
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0,len(text)-maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print('학습할 구문의 수 : ',len(sentences))
print('텍스트를 id벡터로 변환합니다.')

X = np.zeros((len(sentences), maxlen, len(chars)),dtype=np.bool)
y = np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i ,sentence in enumerate(sentences):
    for t,char in enumerate(sentence):
        X[i,t,char_indices[char]] = 1
        y[i,char_indices[next_chars[i]]] = 1

학습할 구문의 수 :  103888
텍스트를 id벡터로 변환합니다.


C:\Users\user\AppData\Local\Temp\ipykernel_8368\4232985311.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(sentences), maxlen, len(chars)),dtype=np.bool)
C:\Users\user\AppData\Local\Temp\ipykernel_8368\4232985311.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences),len(chars)),dtype=np.bool)


In [10]:
# 모델 구축하기
print('모델을 구축합니다')
model = Sequential()
model.add(LSTM(128,input_shape=(maxlen,len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer = optimizer)

# 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 학습시키고 텍스트 생성하기 반복
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('반복=', iteration)
    model.fit(X, y, batch_size=128, epochs=1) #
    # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 다양한 다양성의 문장 생성
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('-- 다양성 = ', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('-- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)
        # 시드를 기반으로 텍스트 자동 생성
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 출력하기
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


모델을 구축합니다

--------------------------------------------------
반복= 1
812/812 [==============================] - 183s 223ms/step - loss: 3.8429

-- 다양성 =  0.2
-- 시드 = " 발을 내디뎠다. 간난할멈은 쩍 벌어"
 발을 내디뎠다. 간난할멈은 쩍 벌어 있었다. 그러나 나가 들어 있었다. 어디 있었다. 그러나 그러나 가지 않고 있었다.
"아이 아니 없이 그 말이 있었다. 그러나 그러나 그러나 그러나 가지 않았다. 마음에 그러나 가지 않고 있었다. 아니 나가 있었다. 그러나 있었다. 그러나 눈에 가서 있었다. "이 아니 아니 나 가지 않는 것이다. 아이 아니 없는 것이다. 아니 말이 있었다. 모양이 있었다. 마음에 가서 그러나 일이 있었다. 것이 있었다. 그러나 모양이다. 그러나 가지 않고 있었다. 그러나 자리 있었다. 그러나 하고 있었다. 그러나 그러나 그러나 가지 않고 있었다. 아니 그러나 그 말이 있었다. 아니 그는 기가 있었다. "그 아니 있었다. 그 말이 있었다. 아닌 그리 않는 것이다. 그 말이 있었다. 자식 그 말이 있었다. 그러나 것이 있었다

-- 다양성 =  0.5
-- 시드 = " 발을 내디뎠다. 간난할멈은 쩍 벌어"
 발을 내디뎠다. 간난할멈은 쩍 벌어 있었다. 평산의 얼굴을 내어 있었다. 일은 모습이다. 그, 이는 일이 있었다. 하는 일이다. 나 그러나 아니다. 불어서 그러나 모양이었다. 방문을 하고 있었다. 최참판댁 하며 
"이 나 가지 않고 있는 기가 오지 않아서 두만네 말이 있었다. 것이 다 소리를 미가 아니었다. 한 상에 아니었다. 소리 들은 아니었다. 웃었다.
"아, 아이 되어서 바라보고 있었다니다. 이 죽을 모양이 없는 것이다. 평산의 용이 치수에 최치수의 바람 일이 없었고 있었다. 그리는 말이 있었다. 그런 것을 있었다. 눈이 아이 있었다.
"아씨가 우리 했다. 그래 어디 수 없고 있는 것이다. 마음이  소리

KeyboardInterrupt: 